In [4]:
# !wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet
# !wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet

In [5]:
# !pip install pyarrow

     |████████████████████████████████| 29.4 MB 606 kB/s eta 0:00:01


In [2]:
import pandas as pd

In [6]:
jan = pd.read_parquet('fhv_tripdata_2021-01.parquet')

Question 1:

In [7]:
jan.shape[0]

1154112

In [8]:
jan.columns

Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number'],
      dtype='object')

In [10]:
jan['duration'] = jan.dropOff_datetime - jan.pickup_datetime

In [11]:
jan.duration = jan.duration.apply(lambda td: td.total_seconds()/60)

Question 2:

In [12]:
jan.duration.mean()

19.167224093791006

In [23]:
jan_limited = jan[(jan.duration >= 1) & (jan.duration <= 60)]

Question 2.5:

In [26]:
jan.shape[0]-jan_limited.shape[0]

44286

In [28]:
jan_limited.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1109826 entries, 0 to 1154111
Data columns (total 8 columns):
 #   Column                  Non-Null Count    Dtype          
---  ------                  --------------    -----          
 0   dispatching_base_num    1109826 non-null  object         
 1   pickup_datetime         1109826 non-null  datetime64[ns] 
 2   dropOff_datetime        1109826 non-null  datetime64[ns] 
 3   PUlocationID            182818 non-null   float64        
 4   DOlocationID            961919 non-null   float64        
 5   SR_Flag                 0 non-null        object         
 6   Affiliated_base_number  1109053 non-null  object         
 7   duration                1109826 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), float64(2), object(3), timedelta64[ns](1)
memory usage: 76.2+ MB


In [ ]:
jan_limited.fillna({"DOlocationID":1}, inplace=True)